<a href="https://colab.research.google.com/github/yzbaya/FAD.Machine-Learning/blob/main/FAD_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [3]:
data = pd.read_csv('historique ventes.csv', encoding='ISO-8859-1')

**Préparation des Données **

In [26]:
# numeric_features = ['NJR','STDA','Ecoulement']
# # columns_to_convert = ['STDA', 'CMJ', 'Ecoulement']
# print(data[numeric_features].dtypes)


NJR           float64
STDA            int64
Ecoulement    float64
dtype: object


In [ ]:
#Nettoyage des données DDV DDA
# data['DDA'] = pd.to_datetime(data['DDA'], errors='coerce', dayfirst=True)
# data['DDV'] = pd.to_datetime(data['DDV'], errors='coerce', dayfirst=True)
# data['month_dernier_vente']=data['DDV'].dt.month
# data['season_dernier_vente']=data['DDV'].dt.month % 12 // 3+1

In [15]:
from sklearn.impute import SimpleImputer
#extract et imputer les colonnes numeriques

numeric_features = ['NJR','CMJ']
print(' colonnes numeriques')
print(data[numeric_features].dtypes)
imputer_numeric=SimpleImputer(strategy='median')
data[numeric_features]=imputer_numeric.fit_transform(data[numeric_features])

# List des colonnes à convertir et à imputer
columns_to_convert = ['Vente 2022','Vente 2023','STDA','Ecoulement']
print('autres colonnes')
print(data[columns_to_convert].dtypes)
def convert_data(data,columns_to_convert):
  for column in columns_to_convert:
    data[column]=data[column].replace({',':''},regex=True)
    data[column]=pd.to_numeric(data[column],errors='coerce')
    imputer_cols=SimpleImputer(strategy='median')
    data[column]=imputer_cols.fit_transform(data[[column]])
convert_data(data,columns_to_convert)
#test
print('colonnes non num')
print(data[columns_to_convert].dtypes)

print("Vérification générale des valeurs manquantes:")
print(data[columns_to_convert].isna().sum())

 colonnes numeriques
NJR    float64
CMJ    float64
dtype: object
autres colonnes
Vente 2022    float64
Vente 2023    float64
STDA          float64
Ecoulement    float64
dtype: object
colonnes non num
Vente 2022    float64
Vente 2023    float64
STDA          float64
Ecoulement    float64
dtype: object
Vérification générale des valeurs manquantes:
Vente 2022    0
Vente 2023    0
STDA          0
Ecoulement    0
dtype: int64


In [16]:
# Sélection et imputation des données catégorielles
categorical_features = ['Pareto']
imputer_mostfrequent = SimpleImputer(strategy='most_frequent')
data[categorical_features] = imputer_mostfrequent.fit_transform(data[categorical_features])

# Mapping importance des pièces
pareto_importance = {
    'AA': 1, 'AB': 2, 'AC': 3, 'AD': 4,
    'BA': 5, 'BB': 6, 'BC': 7, 'BD': 8,
    'CA': 9, 'CB': 10, 'CC': 11, 'CD': 12,
    'DA': 13, 'DB': 14, 'DC': 15, 'DD': 16
}
data['Importance'] = data['Pareto'].map(pareto_importance)

# Encodage one-hot
data = pd.get_dummies(data, columns=['Pareto'], prefix='Pareto')

# Afficher les premières lignes du dataframe pour une vérification visuelle
print(data.head())

KeyError: "None of [Index(['Pareto'], dtype='object')] are in the [columns]"

In [ ]:
from sklearn.model_selection import train_test_split
x=data[numeric_features + categorical_features]
# y=data[target]

KeyError: 'vente_future'